In [ ]:
# Bibliotheken importieren
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
%matplotlib inline


In [ ]:
# Datensatz laden
df = pd.read_csv('data.csv', encoding='ISO-8859-1')
df.head()


In [ ]:
# Überblick über die Daten
df.info()


In [ ]:
# Anzahl fehlender Werte in jeder Spalte anzeigen
df.isnull().sum()


In [ ]:
# Zeilen mit fehlenden Werten entfernen
df = df.dropna()


In [ ]:
# Entferne alle Transaktionen mit negativen oder null Mengen
df = df[df['Quantity'] > 0]
df = df[df['UnitPrice'] > 0]


In [ ]:
# Eine neue Spalte "TotalPrice" berechnen
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']


In [ ]:
# Aktuelles Analyse-Datum festlegen (ein Tag nach dem letzten Transaktionsdatum)
now = datetime(2011, 12, 10)


In [ ]:
# RFM (Recency, Frequency, Monetary) für jeden Kunden berechnen
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (now - pd.to_datetime(x).max()).days,
    'InvoiceNo': 'nunique',
    'TotalPrice': 'sum'
})
rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm.head()


In [ ]:
# RFM-Werte in Scores umwandeln (1-5)
rfm['R'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])
rfm['F'] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm['M'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5])
rfm['RFM_Score'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)
rfm.head()


In [ ]:
# Einfache Segmentierung basierend auf dem RFM-Score
def segment(x):
    if x >= 445:
        return 'Beste Kunden'
    elif x >= 344:
        return 'Loyal'
    elif x >= 233:
        return 'Potenzial'
    else:
        return 'Risiko'

# RFM Score in numerisch konvertieren
rfm['RFM_Score_Numeric'] = rfm['RFM_Score'].astype(int)
rfm['Segment'] = rfm['RFM_Score_Numeric'].apply(segment)
rfm[['RFM_Score', 'Segment']].head()
